<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

In [3]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

In [4]:
print(f'X - {X.shape}\n'\
      f'y - {y.shape}')

X - (506, 13)
y - (506,)


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [5]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
print(f'X_train - {X_train.shape}\n'\
      f'y_train - {y_train.shape}\n'\
      f'X_test - {X_test.shape}\n'\
      f'y_test - {y_test.shape}')

X_train - (404, 13)
y_train - (404,)
X_test - (102, 13)
y_test - (102,)


2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html

In [8]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

In [47]:
reg = LinearRegression().fit(X_train, y_train)
y_reg_test = reg.predict(X_test)
print(f'LinearRegression()\n'\
      f'r2 (test) - {r2_score(y_test, y_reg_test)}')

LinearRegression()
r2 (test) - 0.6687594935356314


In [48]:
reg = Ridge().fit(X_train, y_train)
y_reg_test = reg.predict(X_test)
print(f'Ridge()\n'\
      f'r2 (test) - {r2_score(y_test, y_reg_test)}')

Ridge()
r2 (test) - 0.6662221670168518


In [49]:
reg = Lasso().fit(X_train, y_train)
y_reg_test = reg.predict(X_test)
print(f'Lasso()\n'\
      f'r2 (test) - {r2_score(y_test, y_reg_test)}')

Lasso()
r2 (test) - 0.6671453631686304


In [50]:
def get_r2_score(model, X, y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=RANDOM_STATE)
    
    model.fit(X_train, y_train)
    
    y_pred_train = model.predict(X_train)
    r2_train = r2_score(y_train, y_pred_train)
    
    y_pred_test = model.predict(X_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    return r2_train, r2_test

In [51]:
models = [
    {'name':'LinearRegression', 'model':LinearRegression()}, 
    {'name':'Ridge', 'model':Ridge()}, 
    {'name':'Lasso', 'model':Lasso()}
]

for model in models:
    model_name = model['name']
    model = model['model']
    r2_score_train, r2_score_test = get_r2_score(model, X, y)
    print(f'{model_name}\n'\
          #f'r2 (train) - {r2_score_train}\n'\
          f'r2 (test) - {r2_score_test:.3f}\n')

LinearRegression
r2 (test) - 0.669

Ridge
r2 (test) - 0.666

Lasso
r2 (test) - 0.667



3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV

In [26]:
alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5]

In [35]:
def get_search_cv(model, X, y, grid_search, alphas=alphas, scoring='r2'):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=RANDOM_STATE)
    
    if grid_search:
        params = [{'alpha': alphas}]
        search = GridSearchCV(model(), params, scoring=scoring).fit(X_train, y_train)
        best_alpha = search.best_params_['alpha']
    else:
        search = model(alphas=alphas).fit(X_train, y_train)
        best_alpha = search.alpha_
        
    score_train = search.score(X_train, y_train)
    score_test = search.score(X_test, y_test)
    
    return score_train, score_test, best_alpha

In [36]:
models = [
    {'name':'Ridge', 'clf':Ridge, 'grid_search':True}, 
    {'name':'RidgeCV', 'clf':RidgeCV, 'grid_search':False},
    {'name':'Lasso', 'clf':Lasso, 'grid_search':True},
    {'name':'LassoCV', 'clf':LassoCV, 'grid_search':False}
]

for model in models:
    model_name = model['name']
    clf = model['clf']
    grid_search = model['grid_search']
    
    r2_score_train, r2_score_test, best_alpha = get_search_cv(clf, X, y, grid_search)
    
    print(f'{model_name}\n'\
          #f'r2 (train) - {r2_score_train}\n'\
          f'r2 (test) - {r2_score_test:.6f}\n'\
          f'best alpha - {best_alpha}\n')

Ridge
r2 (test) - 0.668759
best alpha - 1e-05

RidgeCV
r2 (test) - 0.668751
best alpha - 0.01

Lasso
r2 (test) - 0.668760
best alpha - 1e-05

LassoCV
r2 (test) - 0.668760
best alpha - 1e-05



Результат незначительно улучшился. RidgeCV и GridSearchCV(Ridge) выявили разные лучшие коэффициенты регуляризации.

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [9]:
models = [
    {'name':'Ridge after StandardScaler', 'clf':Ridge(), 'scaler':StandardScaler()},
    {'name':'Ridge after MinMaxScaler', 'clf':Ridge(), 'scaler':MinMaxScaler()},
    {'name':'Lasso after StandardScaler', 'clf':Lasso(), 'scaler':StandardScaler()},
    {'name':'Lasso after MinMaxScaler', 'clf':Lasso(), 'scaler':MinMaxScaler()}
]


for model in models:
    model_name = model['name']
    clf = model['clf']
    scaler = model['scaler']
    
    pipe = Pipeline(steps=[('scaler', scaler), ('clf', clf)]).fit(X_train, y_train)
    score = pipe.score(X_test, y_test)
    
    print(f'{model_name}\n'\
          f'{score}\n')

Ridge after StandardScaler
0.6684624359643561

Ridge after MinMaxScaler
0.6764100365423595

Lasso after StandardScaler
0.6239428734251422

Lasso after MinMaxScaler
0.2573921442545194



Лучший результат у Ridge with MinMaxScaler. Lasso with StandardScaler ухудшился. Lasso with MinMaxScaler сильно ухудшился.

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

http://archive.ics.uci.edu/ml/datasets/Adult

In [ ]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [ ]:
data.head()

8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

10. Выберите колонки с числовыми и категориальными переменными.

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.